In [0]:
# GPU로 바꾼후 GPU연동 확인
!nvidia-smi

Sun Jun 14 10:50:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

########학습부분#############



In [0]:
# 구글 드라이브 마운트
# 구글드라이브에 있는 데이터 파일과 kogpt2(구버전)사용
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# 필수 패키지 설치
!pip install -r /content/drive/'My Drive'/NarrativeKoGPT2/requirements.txt

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 9.8MB/s 
  Found existing installation: tokenizers 0.0.11
    Uninstalling tokenizers-0.0.11:
      Successfully uninstalled tokenizers-0.0.11
  Found existing installation: transformers 2.4.0
    Uninstalling transformers-2.4.0:
      Successfully uninstalled transformers-2.4.0


1. 시스템경로 추가

In [0]:
!pip install minegpt2

In [0]:
import os
import sys
sys.path.append('/content/drive/My Drive/')
#print(os.getcwd())
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer

# model 업그레이드로 인해 수정
from minegpt2.kogpt2.utils import get_tokenizer
from minegpt2.kogpt2.utils import download, tokenizer
#from NarrativeKoGPT2.model.torch_gpt2 import GPT2Config, GPT2LMHeadModel
from NarrativeKoGPT2.util.data import NovelDataset
import gluonnlp
from tqdm import tqdm
print(torch.cuda.device_count())  # GPU deviec count check

'''
from NarrativeKoGPT2.util.data import NovelDataset

class NovelDataset(Dataset):
  """web novel dataset"""

  def __init__(self, file_path,vocab,tokenizer):
    self.file_path = file_path
    self.data =[]
    self.vocab =vocab
    self.tokenizer = tokenizer
    file = open(self.file_path, 'r', encoding='utf-8')

    while True:
      line = file.readline()
      if not line:
        break
      toeknized_line = tokenizer(line[:-1])
      index_of_words = vocab[toeknized_line]
      # print(np.shape(index_of_words))
      self.data.append(index_of_words)

    # print(np.shape(self.data))

    file.close()

  def __len__(self):
    return len(self.data)
  def __getitem__(self,index):
    item = self.data[index]
    # print(item)
    return item
'''

1


In [0]:
import subprocess

def get_gpu_memory_map():
    """Get the current gpu usage.

    Returns
    -------
    usage: dict
        Keys are device ids as integers.
        Values are memory usage as integers in MB.
    """
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ], encoding='utf-8')
    # Convert lines into a dictionary
    gpu_memory = [int(x) for x in result.strip().split('\n')]
    gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
    return gpu_memory_map

저기 아래 savepath에서 저장할파일 이름 안바꿔주면 미리해논거날아갈수잇음

소설 파일에 공백인 줄들이 있어서 len(data)!=0 c추가
하라는데로 하면 이런 에러가 뜨기 때문에 공백이있어서그런듯
공백을 없애주거나 공백이면 안하는 식으로 바꿈
수집하는 데이터마다 다른 에러가 뜰수있어서
 수집하고 전처리를 해야할듯...

#gpu로 돌리면 RuntimeError: invalid argument 2: non-empty vector or matrix #expected at /pytorch/aten/src/THCUNN/generic/ClassNLLCriterion.cu:31

파일에 따라서
이런 에러나는데,,,cpu로 돌리면 안남.....
근데 cpu로 돌리면 너무 느림...
그래서 걍 예외처리함...

아래는 버스커버스커 노래 가사. except가 많이 일어나서 loss가 별로 안줄어든듯...
아니면 한 문장이 너무 짧아서 그런걸수도있어서 문장들을 길게 만들고 buskerbusker2.txt로 만들어서 다시 학습해봄
결과적으로 잘됨
아래는 소설에 대한 학습

In [0]:
from NarrativeKoGPT2.kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cache_dir='/content/drive/My Drive/NarrativeKoGPT2/kogpt2/' # KoGPT-2 모델 다운로드 경로
save_path = '/content/drive/My Drive/NarrativeKoGPT2/checkpoint/111.tar' # 학습 후 결과 데이터
data_file_path = '/content/drive/My Drive/NarrativeKoGPT2/data/soseal1.txt' # 학습하고 싶은 데이터
#use_cuda = True # Colab내 GPU 사용을 위한 값



In [0]:

tok_path = get_tokenizer()
model, vocab = get_pytorch_kogpt2_model(ctx,cache_dir)
sentencepieceTokenizer = SentencepieceTokenizer(tok_path)
device=torch.device(ctx)
model.to(device)
model.train()

epoch =200  # 학습 epoch
learning_rate = 1e-5
batch_size = 2
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


novel_dataset = NovelDataset(data_file_path, vocab,sentencepieceTokenizer)
novel_data_loader = DataLoader(novel_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

print('KoGPT-2 Transfer Learning Start')

using cached model
1
using cached model
/content/drive/My Drive/NarrativeKoGPT2/kogpt2/pytorch_kogpt2_676e9bcfa7.params
using cached model
/content/drive/My Drive/NarrativeKoGPT2/kogpt2/kogpt2_news_wiki_ko_cased_818bfa919d.spiece
KoGPT-2 Transfer Learning Start


In [0]:
except_cnt=0
for epoch in range(epoch):
  count = 0
  for data in novel_data_loader:
    try:
      if len(data)!=0:
        optimizer.zero_grad()
        data = torch.stack(data) # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.

        data= data.transpose(1,0)
        data= data.to(ctx)
    
        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss.backward()
        optimizer.step()
        if count %10 ==0:
          print('epoch no.{} train no.{}  loss = {}' . format(epoch, count+1, loss))
          # torch.save(model,save_path+'checkpoint_{}_{}.tar'.format(epoch,count))
          # 추론 및 학습 재개를 위한 일반 체크포인트 저장하기
        if (count >0 and count%100==0) or (len(data) < batch_size):
          torch.save({
            'epoch': epoch,
            'train_no': count,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss':loss
          }, save_path)

        count += 1
    except:
      count +=1
      print("except")


epoch no.0 train no.1  loss = 7.365644454956055
epoch no.0 train no.11  loss = 6.281390190124512
epoch no.0 train no.21  loss = 6.683767795562744
epoch no.1 train no.1  loss = 5.3509955406188965
epoch no.1 train no.11  loss = 5.621183395385742
epoch no.1 train no.21  loss = 4.972174644470215
epoch no.1 train no.31  loss = 4.745198726654053
epoch no.2 train no.1  loss = 5.189884185791016
epoch no.2 train no.11  loss = 5.464982032775879
epoch no.2 train no.21  loss = 5.213677406311035
epoch no.2 train no.31  loss = 4.879258155822754
epoch no.3 train no.1  loss = 5.256425857543945
epoch no.3 train no.11  loss = 4.724671840667725
epoch no.3 train no.21  loss = 4.304839134216309
epoch no.3 train no.31  loss = 4.341354846954346
epoch no.4 train no.1  loss = 5.341293811798096
epoch no.4 train no.11  loss = 4.164826393127441
epoch no.4 train no.21  loss = 4.377991676330566
epoch no.5 train no.1  loss = 3.991142988204956
epoch no.5 train no.11  loss = 4.281426429748535
epoch no.5 train no.21  l

아래는 다운받은 텍스트 소설. 로스가 잘 줄어들고 잘 학습된다